# Importação de Dados CSV para PostgreSQL - PharmaSense AI

Este notebook importa os dados dos arquivos CSV para o banco de dados PostgreSQL:
- `logistica_simulada.csv` → tabela `logistica`
- `demanda_estoque.csv` → tabela `demanda_estoque`

## Configuração do Banco de Dados

As configurações são lidas das variáveis de ambiente (arquivo `.env` ou variáveis do sistema):
- `DB_HOST`
- `DB_PORT`
- `DB_NAME`
- `DB_USER`
- `DB_PASSWORD`

Crie um arquivo `.env` na raiz do projeto com essas variáveis.


## 1. Importação de Bibliotecas


In [6]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
from dotenv import load_dotenv
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

print("Bibliotecas importadas com sucesso!")


Bibliotecas importadas com sucesso!


## 2. Configuração da Conexão


In [7]:
# Verificar se todas as variáveis de ambiente estão definidas
required_vars = ['DB_HOST', 'DB_PORT', 'DB_NAME', 'DB_USER', 'DB_PASSWORD']
missing_vars = [var for var in required_vars if not os.getenv(var)]

if missing_vars:
    print(f"❌ ERRO: Variáveis de ambiente não definidas: {', '.join(missing_vars)}")
    print("\nCrie um arquivo .env na raiz do projeto com:")
    print("DB_HOST=localhost")
    print("DB_PORT=5432")
    print("DB_NAME=pharmasense")
    print("DB_USER=pharmasense")
    print("DB_PASSWORD=pharmasense_pass")
    print("\nOu defina as variáveis de ambiente do sistema.")
    sys.exit(1)

DB_CONFIG = {
    'host': os.getenv('DB_HOST'),
    'port': int(os.getenv('DB_PORT')),
    'database': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD')
}

print(f"Conectando ao banco: {DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}")

try:
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    print("✅ Conexão estabelecida com sucesso!")
except Exception as e:
    print(f"❌ Erro ao conectar ao banco de dados: {e}")
    sys.exit(1)


Conectando ao banco: localhost:5432/pharmasense
✅ Conexão estabelecida com sucesso!


## 3. Carregamento dos Dados CSV


In [8]:
df_logistica = pd.read_csv("../assets/logistica_simulada.csv")
df_logistica["Data"] = pd.to_datetime(df_logistica["Data"])

print(f"✅ Dados de logística carregados: {len(df_logistica):,} registros")
df_logistica.head()


✅ Dados de logística carregados: 290,099 registros


,Data,Estado,Regiao,Rota_ID,Tempo_Resposta_Previsto,Tempo_Resposta_Real,Status,Custo_Logistico_USD,Emissao_CO2_kg
0,2020-02-25,SP,Sudeste,R000001,1.28,1.00,Entregue,132.89,24.33
1,2020-02-28,SP,Sudeste,R000002,1.45,1.00,Entregue,105.48,25.90
2,2020-02-28,SP,Sudeste,R000003,1.27,1.00,Entregue,96.90,31.27
3,2020-03-04,SP,Sudeste,R000004,1.00,1.00,Entregue,119.39,29.42
4,2020-03-05,SP,Sudeste,R000006,1.38,1.04,Entregue,148.26,29.49


In [9]:
df_estoque = pd.read_csv("../assets/demanda_estoque.csv")
df_estoque["Data"] = pd.to_datetime(df_estoque["Data"])

print(f"✅ Dados de estoque carregados: {len(df_estoque):,} registros")
df_estoque.head()


✅ Dados de estoque carregados: 27,029 registros


,Data,Estado,Regiao,Demanda_Diaria,Entregas_Concluidas,Entregas_Atrasadas,Custo_Total_USD,Custo_Medio_USD,Emissao_Total_CO2_kg,Emissao_Media_CO2_kg,...,Demanda_Nao_Atendida,Taxa_Atendimento,Nivel_Servico,Dias_Estoque_Restante,Ponto_Reposicao,Indicador_Estoque_Baixo,Indicador_Stock_Out,Demanda_Acumulada,Stock_Out_Acumulado,Custo_Total_Acumulado
0,2020-03-17,AC,Norte,3,2,0,767.53,383.765,150.84,75.420,...,0,100.0,100.0,12.0,7,0,0,3,0,767.53
1,2020-03-19,AC,Norte,1,1,0,384.83,384.830,78.85,78.850,...,0,100.0,100.0,35.0,7,0,0,4,0,1152.36
2,2020-03-20,AC,Norte,1,1,0,395.09,395.090,79.44,79.440,...,0,100.0,100.0,34.0,7,0,0,5,0,1547.45
3,2020-03-21,AC,Norte,2,2,0,695.54,347.770,160.61,80.305,...,0,100.0,100.0,16.0,7,0,0,7,0,2242.99
4,2020-03-22,AC,Norte,2,2,0,771.64,385.820,141.79,70.895,...,0,100.0,100.0,15.0,7,0,0,9,0,3014.63


## 4. Criação das Tabelas


In [10]:
cur.execute("""
CREATE TABLE IF NOT EXISTS logistica (
    id INTEGER PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
    data DATE NOT NULL,
    estado TEXT NOT NULL,
    regiao TEXT NOT NULL,
    rota_id TEXT NOT NULL,
    tempo_resposta_previsto NUMERIC(5, 2),
    tempo_resposta_real NUMERIC(5, 2),
    status TEXT,
    custo_logistico_usd NUMERIC(10, 2) NOT NULL,
    emissao_co2_kg NUMERIC(8, 2) NOT NULL,
    created_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
);
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS demanda_estoque (
    id INTEGER PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
    data DATE NOT NULL,
    estado TEXT NOT NULL,
    regiao TEXT NOT NULL,
    demanda_diaria INTEGER NOT NULL DEFAULT 0,
    entregas_concluidas INTEGER NOT NULL DEFAULT 0,
    entregas_atrasadas INTEGER NOT NULL DEFAULT 0,
    custo_total_usd NUMERIC(12, 2) NOT NULL,
    custo_medio_usd NUMERIC(10, 2) NOT NULL,
    emissao_total_co2_kg NUMERIC(10, 2) NOT NULL,
    emissao_media_co2_kg NUMERIC(8, 2) NOT NULL,
    tempo_medio_entrega_dias NUMERIC(5, 2) NOT NULL,
    tempo_previsto_medio_dias NUMERIC(5, 2) NOT NULL,
    estoque_inicial INTEGER NOT NULL DEFAULT 0,
    estoque_disponivel INTEGER NOT NULL DEFAULT 0,
    estoque_final INTEGER NOT NULL DEFAULT 0,
    reabastecimento INTEGER NOT NULL DEFAULT 0,
    reabastecimento_chegando INTEGER NOT NULL DEFAULT 0,
    stock_out INTEGER NOT NULL DEFAULT 0,
    demanda_atendida NUMERIC(10, 2) NOT NULL,
    demanda_nao_atendida NUMERIC(10, 2) NOT NULL,
    taxa_atendimento NUMERIC(5, 2) NOT NULL,
    nivel_servico NUMERIC(5, 2) NOT NULL,
    dias_estoque_restante INTEGER NOT NULL DEFAULT 0,
    ponto_reposicao INTEGER NOT NULL DEFAULT 0,
    indicador_estoque_baixo INTEGER NOT NULL DEFAULT 0,
    indicador_stock_out INTEGER NOT NULL DEFAULT 0,
    demanda_acumulada NUMERIC(12, 2) NOT NULL,
    stock_out_acumulado INTEGER NOT NULL DEFAULT 0,
    custo_total_acumulado NUMERIC(12, 2) NOT NULL,
    created_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
);
""")

conn.commit()
print("✅ Tabelas criadas com sucesso!")


✅ Tabelas criadas com sucesso!


## 5. Preparação dos Dados


In [11]:
df_logistica_db = df_logistica.copy()
df_logistica_db["Data"] = df_logistica_db["Data"].dt.date
df_logistica_db = df_logistica_db[df_logistica_db["Status"] != "Em Rota"]

colunas_logistica = ['Data', 'Estado', 'Regiao', 'Rota_ID', 'Tempo_Resposta_Previsto', 
                     'Tempo_Resposta_Real', 'Status', 'Custo_Logistico_USD', 'Emissao_CO2_kg']
df_logistica_insert = df_logistica_db[colunas_logistica]

print(f"Registros preparados: {len(df_logistica_insert):,}")


Registros preparados: 290,099


In [12]:
df_estoque_db = df_estoque.copy()
df_estoque_db["Data"] = df_estoque_db["Data"].dt.date

colunas_estoque = ['Data', 'Estado', 'Regiao', 'Demanda_Diaria', 'Entregas_Concluidas', 
                   'Entregas_Atrasadas', 'Custo_Total_USD', 'Custo_Medio_USD', 
                   'Emissao_Total_CO2_kg', 'Emissao_Media_CO2_kg', 'Tempo_Medio_Entrega_Dias',
                   'Tempo_Previsto_Medio_Dias', 'Estoque_Inicial', 'Estoque_Disponivel',
                   'Estoque_Final', 'Reabastecimento', 'Reabastecimento_Chegando', 'Stock_Out',
                   'Demanda_Atendida', 'Demanda_Nao_Atendida', 'Taxa_Atendimento', 'Nivel_Servico',
                   'Dias_Estoque_Restante', 'Ponto_Reposicao', 'Indicador_Estoque_Baixo',
                   'Indicador_Stock_Out', 'Demanda_Acumulada', 'Stock_Out_Acumulado', 'Custo_Total_Acumulado']
df_estoque_insert = df_estoque_db[colunas_estoque]

print(f"Registros preparados: {len(df_estoque_insert):,}")


Registros preparados: 27,029


## 6. Inserção dos Dados - Logística


In [13]:
cur.execute("SELECT COUNT(*) FROM logistica;")
count_before = cur.fetchone()[0]
print(f"Registros existentes: {count_before:,}")

if count_before == 0:
    columns = ['data', 'estado', 'regiao', 'rota_id', 'tempo_resposta_previsto', 
               'tempo_resposta_real', 'status', 'custo_logistico_usd', 'emissao_co2_kg']
    
    values = [tuple(row) for row in df_logistica_insert.values]
    
    execute_values(
        cur,
        f"INSERT INTO logistica ({', '.join(columns)}) VALUES %s",
        values,
        page_size=10000
    )
    
    conn.commit()
    print(f"✅ {len(df_logistica_insert):,} registros inseridos!")
else:
    print(f"⚠️  Tabela já contém dados. Limpe primeiro se necessário.")


Registros existentes: 0
✅ 290,099 registros inseridos!


## 7. Inserção dos Dados - Estoque


In [14]:
cur.execute("SELECT COUNT(*) FROM demanda_estoque;")
count_before = cur.fetchone()[0]
print(f"Registros existentes: {count_before:,}")

if count_before == 0:
    columns = ['data', 'estado', 'regiao', 'demanda_diaria', 'entregas_concluidas',
               'entregas_atrasadas', 'custo_total_usd', 'custo_medio_usd',
               'emissao_total_co2_kg', 'emissao_media_co2_kg', 'tempo_medio_entrega_dias',
               'tempo_previsto_medio_dias', 'estoque_inicial', 'estoque_disponivel',
               'estoque_final', 'reabastecimento', 'reabastecimento_chegando', 'stock_out',
               'demanda_atendida', 'demanda_nao_atendida', 'taxa_atendimento', 'nivel_servico',
               'dias_estoque_restante', 'ponto_reposicao', 'indicador_estoque_baixo',
               'indicador_stock_out', 'demanda_acumulada', 'stock_out_acumulado', 'custo_total_acumulado']
    
    values = [tuple(row) for row in df_estoque_insert.values]
    
    execute_values(
        cur,
        f"INSERT INTO demanda_estoque ({', '.join(columns)}) VALUES %s",
        values,
        page_size=10000
    )
    
    conn.commit()
    print(f"✅ {len(df_estoque_insert):,} registros inseridos!")
else:
    print(f"⚠️  Tabela já contém dados. Limpe primeiro se necessário.")


Registros existentes: 0
✅ 27,029 registros inseridos!


## 8. Verificação dos Dados


In [17]:
cur.execute("SELECT COUNT(*) FROM logistica;")
count = cur.fetchone()[0]
print(f"✅ Total de registros na tabela logistica: {count:,}")

cur.execute("""
    SELECT 
        MIN(data) as data_min,
        MAX(data) as data_max,
        COUNT(DISTINCT estado) as estados,
        COUNT(DISTINCT regiao) as regioes
    FROM logistica;
""")
stats = cur.fetchone()
print(f"Período: {stats[0]} a {stats[1]}")
print(f"Estados: {stats[2]}, Regiões: {stats[3]}")


✅ Total de registros na tabela logistica: 290,099
Período: 2020-02-25 a 2023-03-18
Estados: 27, Regiões: 5


In [16]:
cur.execute("SELECT COUNT(*) FROM demanda_estoque;")
count = cur.fetchone()[0]
print(f"✅ Total de registros na tabela demanda_estoque: {count:,}")

cur.execute("""
    SELECT 
        MIN(data) as data_min,
        MAX(data) as data_max,
        COUNT(DISTINCT estado) as estados,
        COUNT(DISTINCT regiao) as regioes
    FROM demanda_estoque;
""")
stats = cur.fetchone()
print(f"Período: {stats[0]} a {stats[1]}")
print(f"Estados: {stats[2]}, Regiões: {stats[3]}")


✅ Total de registros na tabela demanda_estoque: 27,029
Período: 2020-02-25 a 2023-03-18
Estados: 27, Regiões: 5
